In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import keras_tuner as kt

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras import Sequential, layers
from keras.layers import Dense, Dropout


In [ ]:
raw_data = pd.read_csv(r"../data/processed_data/df_train.csv", index_col=0)

In [ ]:
x = raw_data.drop(["TARGET"], axis=1)
y = raw_data.filter(["TARGET"], axis=1)

scaler = (
    StandardScaler()
)  # MinMaxScaler often made values = 0.0, which = lost features in training
x_scaled = scaler.fit_transform(x)
y = y.values  # we dont need to scale our binary classification

x_scaled = x_scaled.reshape(
    x_scaled.shape[0], x_scaled.shape[1], 1
)  # data needs to be 3D for a CNN

x_scaled.shape, y.shape

In [ ]:
def model_builder(hp):
    model = Sequential()

    # input layer
    model.add(Dense(units=x_scaled.shape[1], input_shape=(x_scaled.shape[1],)))

    # let the model decide how many layers it wants to have
    for i in range(hp.Int("num_dense_layers", min_value=2, max_value=5, step=1)):
        model.add(
            Dropout(hp.Float("dropout_" + str(i), min_value=0, max_value=0.5, step=0.1))
        )
        model.add(
            Dense(
                units=hp.Int("layer_" + str(i), min_value=32, max_value=2560, step=64),
                activation=hp.Choice("act_" + str(i), ["relu", "sigmoid"]),
            )
        )

    # output shape of the model the same as the number of features
    model.add(
        Dense((1), activation="sigmoid"),
    )

    # compile the model
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-1, 1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=["accuracy"],
    )

    return model


In [ ]:
earlyStop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=4)


In [ ]:
tuner = kt.BayesianOptimization(
    model_builder,
    objective=kt.Objective("accuracy", direction="max"),
    directory=r"../logs",
    project_name="BayOpt_v1.01",
)


In [ ]:
tuner.search(
    x_scaled,
    y,
    epochs=20,
    batch_size=64,
    validation_split=0.2,
    callbacks=[earlyStop],
)


In [ ]:
hpModel = tuner.get_best_models()[0]
hpModel.summary()

In [ ]:
trained_model = hpModel.fit(
    x_scaled, y_scaled, epochs=30, batch_size=32, callbacks=earlyStop
)

In [ ]:
# currently redundant - trying to improve the model architecture
# add CNN layers
for i in range(hp.Int("num_conv_layers", 1, 3)):
    model.add(
        layers.Conv1D(
            filters=hp.Int("filters_" + str(i), 32, 1280, 32),
            kernel_size=hp.Choice("kernel_size_" + str(i), [3, 5, 7]),
            activation="relu",
            input_shape=(x_scaled.shape[1], 1),
        )
    )
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())